In [1]:
!pip install flask_sqlalchemy
!pip install flask_wtf 
!pip install flask_login 
!pip install flask_bcrypt 
from flask import redirect

In [2]:

from flask import Flask, render_template, url_for, session, flash, request, redirect
from datetime import datetime
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
import sqlite3

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///user.db' 
app.config['SECRET_KEY'] = 'thisisasecretkey'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    password = db.Column(db.String(80))
    post = db.relationship ('Post', backref='writer', lazy='dynamic')


class Post(db.Model):
    __tablename__ = 'post'
    id = db.Column(db.Integer, primary_key=True)
    task = db.Column(db.String(25))
    com_date = db.Column(db.String(100))
    post_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)


class NewPostForm(FlaskForm):
    task = StringField("Task", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Task"})
    com_date = StringField("Completion Date", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Completion Date"})
    submit = SubmitField("Add Note")

    
    
class RegisterForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Password"})
    submit = SubmitField("Register")
    
    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")
            
class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")
    

    
class Todo(db.Model):
    __tablename__ = 'todo'
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    com_date = db.Column (db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)

    def __repr__(self):
        return '<Task %r>' % self.id

class NewTodoForm(FlaskForm):
    content = StringField("Content", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Content"})
    com_date = StringField("Completion Date", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Completion Date"})
    submit = SubmitField("Add Note")

    
@app.route("/")
def index():
    tourism = [{'Spot': "Moral Sin Tong", 'Type': "Heritage",'Distance': 0.2},
              {'Spot': "Finger Hill",  'Type': "Mountain", 'Distance': 0.3},
              {'Spot': "Tung Wan Beach",  'Type': "Beach", 'Distance': 0.3}]
    
    df = pd.DataFrame(tourism)
    df.to_csv('tourism.csv',mode='w',index=False)
    df = pd.read_csv("tourism.csv")
    list = df.to_dict('records')
    
    return render_template('index.html', entries = tourism)


@app.route("/blogs")
def blogs():
    tourism1 = [{'Spot': "Moral Sin Tong", 'Type': "Heritage",'Distance': 0.2},
              {'Spot': "Finger Hill",  'Type': "Mountain", 'Distance': 0.3},
              {'Spot': "Tung Wan Beach",  'Type': "Beach", 'Distance': 0.3}]
    
    df = pd.DataFrame(tourism1)
    df.to_csv('tourism1.csv', mode = 'w', index=False)
    df = pd.read_csv("tourism1.csv")
    list = df.to_dict('records')
    return render_template ('blogs.html', entries = tourism1)

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user) 
                return redirect(url_for("new_post"))   
            
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/logout', methods=["GET","POST"])
@login_required
def logout():
    logout_user()
    return redirect(url_for('index'))


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('signup.html',form=form)


@app.route('/new_post', methods=['GET','POST'])
@login_required
def new_post():
    form = NewPostForm()
    if form.validate_on_submit():
        new_post = Post(task=form.task.data, com_date=form.com_date.data, writer=current_user)
        db.session.add(new_post)
        db.session.commit()
        return redirect(url_for('view_posts'))
    return render_template('new_post.html', task='New Task', form=form)


@app.route('/my-posts', methods=['GET','POST'])
@login_required
def view_posts():
    posts = Post.query.filter_by(writer=current_user).all()
    return render_template('my_posts.html', posts=posts, task='My Tasks')


@app.route('/delete-post/<int:id>', methods=['GET',"POST"])
@login_required
def delete_post(id):
    post = Post.query.get_or_404(id)
    db.session.delete(post)
    db.session.commit()
    return redirect(url_for('view_posts'))

@app.route('/update/<int:id>', methods=['GET', 'POST'])
@login_required
def update_post(id):
    post = Post.query.get_or_404(id)

    if request.method == 'POST':
        post.task = request.form.get('Task')
        post.com_date = request.form.get('Completion Date')
        db.session.commit()
        return redirect(url_for('view_posts'))
    return render_template('update.html', post=post, task='My Tasks')
    
    

                           
if __name__ == '__main__':
    app.run('localhost', 9006)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9006/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2021 00:36:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:36:14] "GET /new_post HTTP/1.1" 302 -
127.0.0.1 - - [24/Oct/2021 00:36:14] "GET /login?next=%2Fnew_post HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:21] "GET /new_post HTTP/1.1" 302 -
127.0.0.1 - - [24/Oct/2021 00:37:21] "GET /login?next=%2Fnew_post HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:30] "POST /login?next=%2Fnew_post HTTP/1.1" 302 -
127.0.0.1 - - [24/Oct/2021 00:37:31] "GET /new_post HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:39] "GET /my-posts HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:41] "GET /update/1 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2021 00:37:47] "POST /update/1 HTTP/1.1" 302 -
127.0.0.1 - - [24/Oct/2021 00:37:47] "GET /my-posts HTTP/1.1" 200 -
